# Week 9: Local Motion Planning

## Module III: Planning & Decision Making

### Topics Covered

- Configuration Space and Collision Detection
- Sampling-based Planners (RRT, RRT*)
- Search-based Planners (State Lattice, Hybrid A*)
- Path Smoothing and Optimization
- Real-time Motion Planning

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Understand configuration space and its role in motion planning
2. Implement RRT (Rapidly-Exploring Random Trees) for probabilistic planning
3. Extend RRT to RRT* for asymptotically optimal paths
4. Build state lattice planners with kinematically-feasible motion primitives
5. Perform collision detection for various obstacle shapes
6. Apply path smoothing techniques to improve trajectory quality
7. Compare sampling-based vs search-based planning approaches

---

## Setup

Import required libraries

In [ ]:
# Google Colab optimizations
import gc
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend for lower memory
import matplotlib.pyplot as plt

# Reduce matplotlib memory usage
plt.rcParams['figure.max_open_warning'] = 0
plt.rcParams['agg.path.chunksize'] = 10000

# Memory optimization: Use %matplotlib inline instead of widget for lower RAM usage
# If experiencing memory issues, restart kernel between major cells

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle, Polygon
from matplotlib.collections import LineCollection
import time

# Set random seed for reproducibility
np.random.seed(42)

## 1. Motion Planning Fundamentals

Local motion planning generates **collision-free trajectories** from the current position to a nearby goal, accounting for obstacles and vehicle dynamics.

### Planning Hierarchy

1. **Mission Planning**: Route on road network (A*, Dijkstra)
2. **Behavior Planning**: High-level decisions (FSM, behavior trees)
3. **Local Motion Planning**: Collision-free trajectory generation ← **This week**
4. **Control**: Execute trajectory (PID, MPC)

### Configuration Space (C-space)

**Definition:** The space of all possible robot configurations.

For a 2D vehicle:
- **Work Space**: Physical (x, y) space with obstacles
- **Configuration Space**: (x, y, θ) - position + heading
- **State Space**: (x, y, θ, v) - includes velocity

**C-space Obstacle:** Set of configurations where robot collides with obstacles.

### Motion Planning Problem

**Given:**
- Start configuration $q_{start}$
- Goal configuration $q_{goal}$
- Obstacle set $\mathcal{O}$ in C-space

**Find:**
- Path $\pi: [0, 1] \rightarrow \mathcal{C}_{free}$
- Where $\pi(0) = q_{start}$, $\pi(1) = q_{goal}$
- And $\pi(s) \notin \mathcal{O}$ for all $s \in [0, 1]$

**Objective:** Minimize cost (e.g., path length, time, energy)

### Planning Approaches

| Approach | Method | Completeness | Optimality |
|----------|--------|--------------|------------|
| **Sampling-based** | RRT, RRT* | Probabilistic | RRT*: Asymptotic |
| **Search-based** | A*, State Lattice | Resolution | Yes (discretization) |
| **Optimization** | Trajectory optimization | Local | Local minimum |

### Challenges in Autonomous Driving

1. **High-dimensional state space**: (x, y, θ, v, a)
2. **Nonholonomic constraints**: Can't move sideways
3. **Dynamic obstacles**: Other vehicles, pedestrians
4. **Real-time requirements**: < 100ms planning cycles
5. **Safety**: Must guarantee collision avoidance

## 2. Rapidly-Exploring Random Trees (RRT)

**RRT** is a sampling-based algorithm that builds a tree of collision-free paths by random exploration.

### Algorithm Overview

**Core Idea**: Incrementally grow a tree from $q_{start}$ by randomly sampling the configuration space and connecting to the nearest existing node.

**Algorithm Steps**:

1. **Initialize**: Tree $T$ with root at $q_{start}$
2. **Loop** for $N$ iterations:
   - **Sample**: Random configuration $q_{rand}$
   - **Nearest**: Find nearest node $q_{near}$ in tree to $q_{rand}$
   - **Steer**: Generate new node $q_{new}$ from $q_{near}$ towards $q_{rand}$ (step size $\delta$)
   - **Collision Check**: If path from $q_{near}$ to $q_{new}$ is collision-free:
     - **Add**: $q_{new}$ to tree with edge from $q_{near}$
   - **Goal Check**: If $q_{new}$ is close enough to $q_{goal}$, done!

**Pseudocode**:
```
RRT(q_start, q_goal, n_iterations):
    T.init(q_start)
    
    for i = 1 to n_iterations:
        q_rand = Sample()
        q_near = Nearest(T, q_rand)
        q_new = Steer(q_near, q_rand, step_size)
        
        if CollisionFree(q_near, q_new):
            T.add_vertex(q_new)
            T.add_edge(q_near, q_new)
            
            if Distance(q_new, q_goal) < threshold:
                return Path(T, q_start, q_new)
    
    return FAILURE
```

---

### Key Components

#### **Sampling**

**Uniform Random Sampling**: 
$$q_{rand} \sim \mathcal{U}(\mathcal{C}_{space})$$

**Goal Biasing** (common enhancement):
- With probability $p_{goal}$ (e.g., 0.05), sample $q_{goal}$
- Speeds convergence by occasionally pulling tree towards goal

#### **Nearest Neighbor**

Find $q_{near} = \arg\min_{q \in T} d(q, q_{rand})$

**Distance Metrics**:
- Euclidean: $d(q_1, q_2) = \sqrt{(x_2-x_1)^2 + (y_2-y_1)^2}$
- For configurations with heading: $d = \sqrt{(x_2-x_1)^2 + (y_2-y_1)^2 + w(\theta_2-\theta_1)^2}$

**Data Structures**:
- Naive: Linear search $O(n)$
- **KD-Tree**: $O(\log n)$ average case

#### **Steering**

Generate $q_{new}$ at most $\delta$ distance from $q_{near}$ towards $q_{rand}$:

$$q_{new} = q_{near} + \min\left(\delta, d(q_{near}, q_{rand})\right) \cdot \frac{q_{rand} - q_{near}}{d(q_{near}, q_{rand})}$$

**Purpose**: 
- Limits tree growth per iteration
- Ensures dense coverage

#### **Collision Checking**

Check if path from $q_{near}$ to $q_{new}$ intersects any obstacle.

**Methods**:
- **Discretization**: Sample points along path
- **Line-obstacle intersection**: Geometric tests

---

### Properties

**Probabilistic Completeness**: 
- If a solution exists, RRT will find it with probability approaching 1 as iterations → ∞

**NOT Optimal**:
- First solution found is typically far from optimal
- No guarantee on path quality

**Space Exploration**:
- **Voronoi Bias**: Tends to explore large empty regions
- Good for finding solutions quickly
- Not focused on optimization

---

### RRT Variants

**RRT-Connect** (Bidirectional):
- Grow two trees: one from start, one from goal
- Try to connect them
- **2-4× faster** than single-tree RRT

**Goal-Biased RRT**:
- Sample goal with probability $p_{goal}$ (e.g., 5%)
- Faster convergence in simple environments

**Kinodynamic RRT**:
- Respects dynamics (acceleration limits, etc.)
- Uses control inputs instead of straight-line steering

---

### Limitations

1. **No optimality**: Path quality can be poor
2. **Parameter tuning**: Step size $\delta$ affects performance
3. **High dimensions**: Becomes slower in high-D spaces
4. **No smoothness**: Paths have sharp turns

**Solution**: Use RRT* for optimality

In [ ]:
# Complete RRT Implementation

class RRTNode:
    """Node in RRT tree."""
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.parent = None
        self.cost = 0.0  # Cost from start (for RRT*)
    
    def __repr__(self):
        return f"Node({self.x:.2f}, {self.y:.2f})"


class Environment:
    """2D environment with rectangular obstacles."""
    
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.obstacles = []  # List of (x, y, width, height)
    
    def add_obstacle(self, x, y, width, height):
        """Add rectangular obstacle."""
        self.obstacles.append((x, y, width, height))
    
    def is_collision(self, x, y, margin=0.5):
        """Check if point (x, y) collides with any obstacle."""
        # Check bounds
        if x < 0 or x > self.width or y < 0 or y > self.height:
            return True
        
        # Check obstacles
        for ox, oy, ow, oh in self.obstacles:
            if (ox - margin <= x <= ox + ow + margin and
                oy - margin <= y <= oy + oh + margin):
                return True
        return False
    
    def is_path_collision_free(self, x1, y1, x2, y2, resolution=0.5):
        """Check if line segment from (x1,y1) to (x2,y2) is collision-free."""
        distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        n_points = int(distance / resolution) + 1
        
        for i in range(n_points + 1):
            t = i / n_points if n_points > 0 else 0
            x = x1 + t * (x2 - x1)
            y = y1 + t * (y2 - y1)
            
            if self.is_collision(x, y):
                return False
        return True


class RRT:
    """RRT (Rapidly-Exploring Random Trees) planner."""
    
    def __init__(self, start, goal, environment, 
                 max_iterations=300, step_size=2.0, goal_sample_rate=0.05):
        """
        Args:
            start: (x, y) start position
            goal: (x, y) goal position
            environment: Environment object
            max_iterations: Maximum planning iterations
            step_size: Maximum distance to extend tree
            goal_sample_rate: Probability of sampling goal
        """
        self.start = RRTNode(start[0], start[1])
        self.goal = RRTNode(goal[0], goal[1])
        self.env = environment
        self.max_iterations = max_iterations
        self.step_size = step_size
        self.goal_sample_rate = goal_sample_rate
        
        self.nodes = [self.start]
        self.goal_threshold = 2.0
        
    def sample(self):
        """Sample random point in configuration space."""
        if np.random.rand() < self.goal_sample_rate:
            # Goal biasing
            return self.goal.x, self.goal.y
        else:
            # Uniform random
            x = np.random.uniform(0, self.env.width)
            y = np.random.uniform(0, self.env.height)
            return x, y
    
    def nearest(self, x, y):
        """Find nearest node in tree to (x, y)."""
        min_dist = float('inf')
        nearest_node = None
        
        for node in self.nodes:
            dist = np.sqrt((node.x - x)**2 + (node.y - y)**2)
            if dist < min_dist:
                min_dist = dist
                nearest_node = node
        
        return nearest_node
    
    def steer(self, from_node, to_x, to_y):
        """Create new node from from_node towards (to_x, to_y)."""
        # Direction vector
        dx = to_x - from_node.x
        dy = to_y - from_node.y
        distance = np.sqrt(dx**2 + dy**2)
        
        # Limit distance to step_size
        if distance > self.step_size:
            dx = dx / distance * self.step_size
            dy = dy / distance * self.step_size
        
        new_node = RRTNode(from_node.x + dx, from_node.y + dy)
        new_node.parent = from_node
        new_node.cost = from_node.cost + np.sqrt(dx**2 + dy**2)
        
        return new_node
    
    def plan(self):
        """Run RRT algorithm."""
        for i in range(self.max_iterations):
            # Sample
            x_rand, y_rand = self.sample()
            
            # Nearest
            nearest_node = self.nearest(x_rand, y_rand)
            
            # Steer
            new_node = self.steer(nearest_node, x_rand, y_rand)
            
            # Collision check
            if self.env.is_path_collision_free(
                nearest_node.x, nearest_node.y,
                new_node.x, new_node.y
            ):
                self.nodes.append(new_node)
                
                # Check if reached goal
                dist_to_goal = np.sqrt((new_node.x - self.goal.x)**2 + 
                                      (new_node.y - self.goal.y)**2)
                
                if dist_to_goal < self.goal_threshold:
                    # Connect to goal
                    if self.env.is_path_collision_free(
                        new_node.x, new_node.y,
                        self.goal.x, self.goal.y
                    ):
                        self.goal.parent = new_node
                        self.goal.cost = new_node.cost + dist_to_goal
                        self.nodes.append(self.goal)
                        return self.extract_path(), i + 1
        
        # Failed to find path
        return None, self.max_iterations
    
    def extract_path(self):
        """Extract path from start to goal."""
        if self.goal not in self.nodes:
            return None
        
        path = []
        current = self.goal
        
        while current is not None:
            path.append((current.x, current.y))
            current = current.parent
        
        path.reverse()
        return path


# Create environment
env = Environment(width=50, height=50)

# Add obstacles
env.add_obstacle(10, 10, 5, 15)
env.add_obstacle(25, 0, 5, 20)
env.add_obstacle(25, 25, 5, 15)
env.add_obstacle(35, 15, 10, 5)
env.add_obstacle(15, 30, 15, 5)

# Set start and goal
start = (5, 5)
goal = (45, 45)

# Run RRT
np.random.seed(42)
rrt = RRT(start, goal, env, max_iterations=300, step_size=2.0)
path, iterations = rrt.plan()

# Simple visualization - path only
fig, ax = plt.subplots(1, 1, figsize=(10, 8))

# Draw obstacles
for ox, oy, ow, oh in env.obstacles:
    rect = Rectangle((ox, oy), ow, oh, facecolor='gray', edgecolor='black', linewidth=2)
    ax.add_patch(rect)

# Draw path
if path:
    path_x, path_y = zip(*path)
    ax.plot(path_x, path_y, 'b-', linewidth=3, label='RRT Path', zorder=5)
    ax.scatter(path_x, path_y, c='blue', s=50, zorder=6)

    path_length = sum(np.sqrt((path[i+1][0]-path[i][0])**2+(path[i+1][1]-path[i][1])**2)
                     for i in range(len(path)-1))
    title = f'RRT Path (Colab Optimized)\nLength: {path_length:.2f}, Nodes: {len(rrt.nodes)}'
else:
    title = 'RRT: Path Not Found'

# Start and goal
ax.scatter([start[0]], [start[1]], c='green', s=300, marker='o',
          edgecolors='black', linewidths=2, label='Start', zorder=10)
ax.scatter([goal[0]], [goal[1]], c='red', s=300, marker='*',
          edgecolors='black', linewidths=2, label='Goal', zorder=10)

ax.set_xlim([0, env.width])
ax.set_ylim([0, env.height])
ax.set_xlabel('X', fontsize=12)
ax.set_ylabel('Y', fontsize=12)
ax.set_title(title, fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_aspect('equal')

plt.tight_layout()
plt.show()
plt.close(fig)

# Results
print("="*70)
print("RRT RESULTS")
print("="*70)
print(f"Iterations: {iterations}, Nodes: {len(rrt.nodes)}")
if path:
    print(f"Path length: {path_length:.2f}, Waypoints: {len(path)}")
print("="*70)

# Cleanup
del rrt, path
gc.collect()


## 3. RRT* - Optimal RRT

RRT* extends RRT with **rewiring** to achieve asymptotically optimal paths.

### Key Differences from RRT

| Aspect | RRT | RRT* |
|--------|-----|------|
| **Path Quality** | First solution (suboptimal) | Converges to optimal |
| **Rewiring** | No | Yes |
| **Nearest** | 1 nearest node | Nodes within radius |
| **Complexity** | $O(n)$ | $O(n \log n)$ |
| **Time** | Faster for first solution | Better final solution |

### Additional Steps

After adding $q_{new}$ to tree:

1. **Choose Best Parent**: Among nearby nodes, select parent that minimizes cost to $q_{new}$
2. **Rewire Tree**: For each nearby node, check if routing through $q_{new}$ reduces cost

### Algorithm Enhancement

```python
# After collision-free edge is found:
1. near_nodes = Near(T, q_new, radius)
2. min_cost_parent = ChooseBestParent(near_nodes, q_new)
3. T.add_vertex(q_new)
4. T.add_edge(min_cost_parent, q_new)
5. Rewire(T, near_nodes, q_new)  # Update tree structure
```

### Shrinking Radius

For asymptotic optimality, use:
$$r_n = \min\left\{ \gamma \left(\frac{\log n}{n}\right)^{1/d}, \delta \right\}$$

Where:
- $n$ = number of nodes
- $d$ = dimension of C-space
- $\gamma$ = constant (typically 10-30)
- $\delta$ = max step size

### Complexity

- **RRT**: $O(n)$ per iteration
- **RRT***: $O(n \log n)$ per iteration (due to near neighbor search)

**Trade-off**: Slower initial solution, better final path quality

In [ ]:
# NOTE: Run cell 5 first to define the RRT base class
# This cell extends RRT with optimal rewiring

# RRT* Implementation with Rewiring

class RRTStar(RRT):
    """RRT* - Asymptotically Optimal RRT."""
    
    def __init__(self, start, goal, environment, 
                 max_iterations=250, step_size=2.0, goal_sample_rate=0.05,
                 rewire_radius_factor=20.0):
        """
        Args:
            rewire_radius_factor: Gamma parameter for shrinking radius
        """
        super().__init__(start, goal, environment, max_iterations, step_size, goal_sample_rate)
        self.rewire_radius_factor = rewire_radius_factor
        
    def search_radius(self):
        """Calculate search radius for near neighbors (shrinking with tree size)."""
        n = len(self.nodes)
        d = 2  # Dimension of C-space
        
        # r_n = min(gamma * (log(n) / n)^(1/d), step_size)
        if n > 1:
            radius = min(
                self.rewire_radius_factor * (np.log(n) / n)**(1/d),
                self.step_size
            )
        else:
            radius = self.step_size
        
        return radius
    
    def near(self, x, y, radius):
        """Find all nodes within radius of (x, y)."""
        near_nodes = []
        
        for node in self.nodes:
            dist = np.sqrt((node.x - x)**2 + (node.y - y)**2)
            if dist <= radius:
                near_nodes.append(node)
        
        return near_nodes
    
    def choose_best_parent(self, near_nodes, new_node):
        """Choose parent that minimizes cost to new_node."""
        if not near_nodes:
            return None
        
        min_cost = float('inf')
        best_parent = None
        
        for near_node in near_nodes:
            # Cost to reach new_node through near_node
            edge_cost = np.sqrt((near_node.x - new_node.x)**2 + 
                               (near_node.y - new_node.y)**2)
            total_cost = near_node.cost + edge_cost
            
            # Check if path is collision-free
            if (total_cost < min_cost and 
                self.env.is_path_collision_free(near_node.x, near_node.y,
                                               new_node.x, new_node.y)):
                min_cost = total_cost
                best_parent = near_node
        
        if best_parent:
            new_node.parent = best_parent
            new_node.cost = min_cost
        
        return best_parent
    
    def rewire(self, near_nodes, new_node):
        """Rewire tree to reduce costs through new_node."""
        for near_node in near_nodes:
            # Skip if same node
            if near_node == new_node:
                continue
            
            # Cost to reach near_node through new_node
            edge_cost = np.sqrt((new_node.x - near_node.x)**2 + 
                               (new_node.y - near_node.y)**2)
            new_cost = new_node.cost + edge_cost
            
            # If cheaper and collision-free, rewire
            if (new_cost < near_node.cost and
                self.env.is_path_collision_free(new_node.x, new_node.y,
                                               near_node.x, near_node.y)):
                # Rewire: update parent
                near_node.parent = new_node
                near_node.cost = new_cost
                
                # Propagate cost updates to children
                self.propagate_cost_to_children(near_node)
    
    def propagate_cost_to_children(self, parent_node):
        """Update costs for all descendants of parent_node."""
        for node in self.nodes:
            if node.parent == parent_node:
                edge_cost = np.sqrt((parent_node.x - node.x)**2 + 
                                   (parent_node.y - node.y)**2)
                node.cost = parent_node.cost + edge_cost
                self.propagate_cost_to_children(node)
    
    def plan(self):
        """Run RRT* algorithm with rewiring."""
        best_path_cost = float('inf')
        best_path = None
        cost_history = []
        
        for i in range(self.max_iterations):
            # Sample
            x_rand, y_rand = self.sample()
            
            # Nearest
            nearest_node = self.nearest(x_rand, y_rand)
            
            # Steer
            new_node = self.steer(nearest_node, x_rand, y_rand)
            
            # Find near nodes
            radius = self.search_radius()
            near_nodes = self.near(new_node.x, new_node.y, radius)
            
            # Choose best parent (RRT* modification)
            best_parent = self.choose_best_parent(near_nodes, new_node)
            
            if best_parent is not None:
                # Add to tree
                self.nodes.append(new_node)
                
                # Rewire tree (RRT* modification)
                self.rewire(near_nodes, new_node)
                
                # Check if reached goal
                dist_to_goal = np.sqrt((new_node.x - self.goal.x)**2 + 
                                      (new_node.y - self.goal.y)**2)
                
                if dist_to_goal < self.goal_threshold:
                    if self.env.is_path_collision_free(new_node.x, new_node.y,
                                                       self.goal.x, self.goal.y):
                        # Update goal's parent if this path is better
                        path_cost = new_node.cost + dist_to_goal
                        
                        if path_cost < best_path_cost:
                            self.goal.parent = new_node
                            self.goal.cost = path_cost
                            best_path_cost = path_cost
                            best_path = self.extract_path()
                            
                            if self.goal not in self.nodes:
                                self.nodes.append(self.goal)
            
            # Record best cost so far
            if best_path:
                cost_history.append(best_path_cost)
            else:
                cost_history.append(float('inf'))
        
        return best_path, self.max_iterations, cost_history


# Compare RRT vs RRT*
np.random.seed(42)
rrt = RRT(start, goal, env, max_iterations=250, step_size=2.0)
rrt_path, rrt_iterations = rrt.plan()

np.random.seed(42)
rrt_star = RRTStar(start, goal, env, max_iterations=250, step_size=2.0, rewire_radius_factor=15.0)
rrt_star_path, rrt_star_iterations, cost_history = rrt_star.plan()

# Calculate lengths
if rrt_path:
    rrt_len = sum(np.sqrt((rrt_path[i+1][0]-rrt_path[i][0])**2+(rrt_path[i+1][1]-rrt_path[i][1])**2) for i in range(len(rrt_path)-1))
else:
    rrt_len = float('inf')

if rrt_star_path:
    rrt_star_len = sum(np.sqrt((rrt_star_path[i+1][0]-rrt_star_path[i][0])**2+(rrt_star_path[i+1][1]-rrt_star_path[i][1])**2) for i in range(len(rrt_star_path)-1))
else:
    rrt_star_len = float('inf')

improvement = ((rrt_len - rrt_star_len) / rrt_len * 100) if rrt_len != float('inf') else 0

# Simple 1x2 visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# RRT path
for ox, oy, ow, oh in env.obstacles:
    ax1.add_patch(Rectangle((ox, oy), ow, oh, facecolor='gray', edgecolor='black', linewidth=2))
if rrt_path:
    px, py = zip(*rrt_path)
    ax1.plot(px, py, 'b-', linewidth=3, label='RRT Path')
ax1.scatter([start[0]], [start[1]], c='green', s=300, marker='o', edgecolors='black', linewidths=2, label='Start', zorder=10)
ax1.scatter([goal[0]], [goal[1]], c='red', s=300, marker='*', edgecolors='black', linewidths=2, label='Goal', zorder=10)
ax1.set_xlim([0, env.width])
ax1.set_ylim([0, env.height])
ax1.set_title('RRT - Length: {:.2f}, Nodes: {}'.format(rrt_len, len(rrt.nodes)), fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_aspect('equal')

# RRT* path
for ox, oy, ow, oh in env.obstacles:
    ax2.add_patch(Rectangle((ox, oy), ow, oh, facecolor='gray', edgecolor='black', linewidth=2))
if rrt_star_path:
    px, py = zip(*rrt_star_path)
    ax2.plot(px, py, 'purple', linewidth=3, label='RRT* Path')
ax2.scatter([start[0]], [start[1]], c='green', s=300, marker='o', edgecolors='black', linewidths=2, label='Start', zorder=10)
ax2.scatter([goal[0]], [goal[1]], c='red', s=300, marker='*', edgecolors='black', linewidths=2, label='Goal', zorder=10)
ax2.set_xlim([0, env.width])
ax2.set_ylim([0, env.height])
ax2.set_title('RRT* - Length: {:.2f}, Nodes: {}'.format(rrt_star_len, len(rrt_star.nodes)), fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_aspect('equal')

plt.tight_layout()
plt.show()
plt.close(fig)

# Results
print("="*70)
print("RRT vs RRT* COMPARISON (Colab Optimized)")
print("="*70)
print("RRT:   Length={:.2f}, Nodes={}".format(rrt_len, len(rrt.nodes)))
print("RRT*:  Length={:.2f}, Nodes={}".format(rrt_star_len, len(rrt_star.nodes)))
print("Improvement: {:.1f}% shorter path".format(improvement))
print("="*70)

# Clear memory
del rrt, rrt_star, rrt_path, rrt_star_path, cost_history
gc.collect()


## 4. State Lattice Planning

State lattice planning uses a **discrete graph** of kinematically-feasible motions precomputed for the vehicle.

### Key Idea

Instead of random sampling, use a **library of motion primitives** that respect vehicle constraints (e.g., turning radius, acceleration limits).

### Motion Primitives

Precomputed trajectories from a reference state:
- Straight
- Left turn (various radii)
- Right turn (various radii)
- Lane change left
- Lane change right

### State Lattice Construction

1. **Discretize state space**: (x, y, θ) grid
2. **For each state, apply motion primitives** to reach neighboring states
3. **Build graph** of reachable states
4. **Search graph** using A* or Dijkstra

### Advantages

- **Kinematically feasible**: Paths respect vehicle dynamics
- **Optimal on lattice**: A* guarantees shortest path on graph
- **Fast**: Graph search is efficient
- **Predictable**: Deterministic results

### Disadvantages

- **Resolution completeness**: Only finds solutions on the lattice
- **Memory intensive**: Must store all primitives
- **Requires tuning**: Lattice resolution vs. computation

### Applications

- **Highway driving**: Structured environments with lanes
- **Parking**: Low-speed maneuvering
- **Urban navigation**: Road-constrained motion

## Exercises

### Exercise 1: Implement RRT*

Extend the RRT implementation to add the rewiring step for path optimization.

**Tasks:**
1. Implement the `Near()` function to find nodes within radius
2. Add "Choose Best Parent" logic
3. Add "Rewire Tree" logic
4. Compare path quality (cost) between RRT and RRT*
5. Plot cost vs. iterations to show convergence

**Hint:** Use shrinking radius $r = \gamma (\log n / n)^{1/2}$ where $\gamma \approx 10-20$

In [ ]:
# Exercise 3 - Template

class DynamicObstacle:
    """Moving obstacle"""
    def __init__(self, x0, y0, vx, vy, radius):
        self.x0 = x0
        self.y0 = y0
        self.vx = vx
        self.vy = vy
        self.radius = radius
    
    def position_at_time(self, t):
        """Return position at time t"""
        return (self.x0 + self.vx * t, self.y0 + self.vy * t)

# TODO: Extend the RRT class from Section 2 to handle dynamic obstacles
# 
# Hints:
# 1. Add time as a state variable: RRTNode should store (x, y, t)
# 2. Modify collision checking to check space-time collision
# 3. When steering, also advance time proportional to distance traveled
# 4. Check if path intersects with moving obstacle trajectory
#
# Example approach:
# class DynamicRRT(RRT):  # Extend RRT from above
#     def __init__(self, environment, start, goal, dynamic_obstacles, **kwargs):
#         super().__init__(environment, start, goal, **kwargs)
#         self.dynamic_obstacles = dynamic_obstacles
#     
#     def is_collision_at_time(self, x, y, t):
#         """Check collision with moving obstacles at time t"""
#         # Check static obstacles (from parent class)
#         if self.env.is_collision(x, y):
#             return True
#         
#         # Check dynamic obstacles
#         for obs in self.dynamic_obstacles:
#             obs_x, obs_y = obs.position_at_time(t)
#             dist = np.sqrt((x - obs_x)**2 + (y - obs_y)**2)
#             if dist < obs.radius:
#                 return True
#         return False

# Your implementation here

## References and Additional Resources

### Core Textbooks

1. **Planning Algorithms** by LaValle (2006)
   - Chapters 5-6: Sampling-based Planning
   - Comprehensive coverage of RRT, RRT*
   - Free online: http://planning.cs.uiuc.edu/

2. **Robotics: Modelling, Planning and Control** by Siciliano et al. (2009)
   - Chapter 6: Motion Planning

3. **Principles of Robot Motion** by Choset et al. (2005)
   - Path planning algorithms
   - Configuration space concepts

### Key Papers

1. **RRT**
   - LaValle (1998) - "Rapidly-Exploring Random Trees: A New Tool for Path Planning"
   - Original RRT paper

2. **RRT***
   - Karaman & Frazzoli (2011) - "Sampling-based Algorithms for Optimal Motion Planning"
   - Proves asymptotic optimality

3. **State Lattice**
   - Pivtoraiko & Kelly (2005) - "Generating Near Minimal Spanning Control Sets for Constrained Motion Planning"
   - McNaughton et al. (2011) - "Motion Planning for Autonomous Driving with a Conformal Spatiotemporal Lattice"

4. **Dynamic Environments**
   - Ferguson et al. (2008) - "Replanning with RRTs"
   - Anytime Dynamic A*

### Algorithms & Variants

1. **RRT Variants**
   - RRT-Connect: Bidirectional RRT
   - RRT*-Smart: Improved sampling
   - Informed RRT*: Ellipsoidal sampling
   - Kinodynamic-RRT: Respects dynamics

2. **Other Sampling Methods**
   - PRM (Probabilistic Roadmap)
   - EST (Expansive Space Trees)
   - SBL (Single-Query Bidirectional Lazy Planner)

3. **Search-Based Methods**
   - Field D*: Interpolated paths
   - Hybrid A*: Continuous state space
   - Theta*: Any-angle planning

### Software & Libraries

1. **OMPL (Open Motion Planning Library)**
   - C++ library with Python bindings
   - Implements RRT, RRT*, PRM, etc.
   - https://ompl.kavrakilab.org/

2. **Python Robotics**
   - Educational implementations
   - https://github.com/AtsushiSakai/PythonRobotics

3. **MoveIt** (ROS)
   - Motion planning framework
   - Industrial manipulation and mobile robots

### Applications in Autonomous Vehicles

1. **Highway Driving**
   - Lattice planners for lane keeping/changing
   - Polynomial trajectory optimization

2. **Urban Driving**
   - Hybrid approaches (lattice + optimization)
   - Interaction-aware planning

3. **Parking**
   - RRT for maneuvering in tight spaces
   - Reeds-Shepp curves

### Advanced Topics

1. **Kinodynamic Planning**
   - Incorporate acceleration/velocity limits
   - Respect vehicle dynamics

2. **Uncertainty**
   - Chance-constrained RRT
   - Robust trajectory optimization

3. **Learning-Based**
   - Neural Motion Planning
   - Learning sampling distributions
   - Imitation learning for primitives

4. **Multi-Query Planning**
   - PRM for multiple start-goal queries
   - Roadmap maintenance

### Practical Considerations

1. **Real-Time Planning**
   - Anytime algorithms
   - Incremental replanning
   - Computing budgets (< 100ms)

2. **Safety**
   - Guaranteed collision avoidance
   - Safe corridors
   - Emergency maneuvers

3. **Smoothness**
   - Path smoothing post-processing
   - Bezier/spline fitting
   - Jerk minimization

---

## Summary

This notebook covered local motion planning for autonomous vehicles:

**Key Concepts:**
- Configuration space and collision detection
- RRT for probabilistic complete planning
- RRT* for asymptotically optimal paths
- State lattice for kinematically-feasible planning

**Practical Skills:**
- Implementing sampling-based planners
- Building collision-free paths
- Understanding trade-offs (completeness vs. optimality vs. speed)
- Handling complex environments with obstacles

**Next Steps:**
1. Study trajectory optimization (MPC, numerical optimization)
2. Learn about prediction-aware planning
3. Explore learning-based motion planning
4. Implement real-time replanning strategies

---

# Exercise Solutions

Below are complete solutions and enhancements to the exercises. The main RRT* implementation was already shown in cell 7 above. These solutions provide additional analysis and improvements.

## Solution to Exercise 1: RRT* Analysis and Enhancements

The RRT* algorithm was already implemented in cell 7 above. This solution provides:
1. Detailed convergence analysis
2. Parameter sensitivity study
3. Performance comparison with different rewire radius values
4. Cost convergence visualization

In [ ]:
# Exercise 1 Solution: RRT* Convergence Analysis and Parameter Study

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import time

def analyze_rrt_star_convergence():
    """
    Comprehensive analysis of RRT* showing:
    - Cost convergence over iterations
    - Effect of rewire radius parameter (gamma)
    - Comparison with RRT
    - Statistical analysis over multiple runs
    """
    
    # Use the same environment as before
    env = Environment(width=50, height=50)
    env.add_obstacle(10, 10, 5, 15)
    env.add_obstacle(25, 0, 5, 20)
    env.add_obstacle(25, 25, 5, 15)
    env.add_obstacle(35, 15, 10, 5)
    env.add_obstacle(15, 30, 15, 5)
    
    start = (5, 5)
    goal = (45, 45)
    
    print("Running RRT* Convergence Analysis...")
    print("="*70)
    
    # Part 1: Single run with detailed cost tracking
    print("\n1. Detailed convergence tracking...")
    np.random.seed(42)
    rrt_star = RRTStar(start, goal, env, max_iterations=500, 
                       step_size=2.0, rewire_radius_factor=15.0)
    path, iterations, cost_history = rrt_star.plan()
    
    if path:
        final_cost = sum(np.sqrt((path[i+1][0]-path[i][0])**2 + 
                                 (path[i+1][1]-path[i][1])**2) 
                        for i in range(len(path)-1))
        print(f"   Final path cost: {final_cost:.2f}")
        print(f"   Number of nodes: {len(rrt_star.nodes)}")
        print(f"   Path waypoints: {len(path)}")
    
    # Part 2: Compare different gamma values
    print("\n2. Testing different rewire radius parameters (gamma)...")
    gamma_values = [5.0, 10.0, 15.0, 20.0, 30.0]
    gamma_results = []
    
    for gamma in gamma_values:
        np.random.seed(42)  # Same random seed for fair comparison
        rrt_test = RRTStar(start, goal, env, max_iterations=300,
                          step_size=2.0, rewire_radius_factor=gamma)
        test_path, _, test_cost_hist = rrt_test.plan()
        
        if test_path:
            path_len = sum(np.sqrt((test_path[i+1][0]-test_path[i][0])**2 + 
                                   (test_path[i+1][1]-test_path[i][1])**2)
                          for i in range(len(test_path)-1))
            gamma_results.append({
                'gamma': gamma,
                'path_length': path_len,
                'nodes': len(rrt_test.nodes),
                'cost_history': test_cost_hist
            })
            print(f"   Gamma={gamma:5.1f}: Path length={path_len:.2f}, Nodes={len(rrt_test.nodes)}")
    
    # Part 3: Multiple runs for statistical analysis
    print("\n3. Running 10 trials for statistical analysis...")
    num_trials = 10
    rrt_results = []
    rrt_star_results = []
    
    for trial in range(num_trials):
        seed = 100 + trial
        
        # RRT
        np.random.seed(seed)
        rrt_trial = RRT(start, goal, env, max_iterations=300, step_size=2.0)
        rrt_path, _ = rrt_trial.plan()
        if rrt_path:
            rrt_len = sum(np.sqrt((rrt_path[i+1][0]-rrt_path[i][0])**2 + 
                                  (rrt_path[i+1][1]-rrt_path[i][1])**2)
                         for i in range(len(rrt_path)-1))
            rrt_results.append(rrt_len)
        
        # RRT*
        np.random.seed(seed)
        rrt_star_trial = RRTStar(start, goal, env, max_iterations=300, 
                                 step_size=2.0, rewire_radius_factor=15.0)
        rrt_star_path, _, _ = rrt_star_trial.plan()
        if rrt_star_path:
            rrt_star_len = sum(np.sqrt((rrt_star_path[i+1][0]-rrt_star_path[i][0])**2 + 
                                       (rrt_star_path[i+1][1]-rrt_star_path[i][1])**2)
                              for i in range(len(rrt_star_path)-1))
            rrt_star_results.append(rrt_star_len)
    
    print(f"   RRT:   Mean={np.mean(rrt_results):.2f}, Std={np.std(rrt_results):.2f}")
    print(f"   RRT*:  Mean={np.mean(rrt_star_results):.2f}, Std={np.std(rrt_star_results):.2f}")
    print(f"   Improvement: {(1 - np.mean(rrt_star_results)/np.mean(rrt_results))*100:.1f}%")
    
    # Visualization
    fig = plt.figure(figsize=(18, 12))
    
    # Plot 1: Cost convergence over iterations
    ax1 = plt.subplot(2, 3, 1)
    # Filter out infinite costs for better visualization
    valid_costs = [c for c in cost_history if c != float('inf')]
    if valid_costs:
        iterations_to_solution = len(cost_history) - len(valid_costs)
        ax1.plot(range(iterations_to_solution, len(cost_history)), valid_costs, 
                'b-', linewidth=2, label='Path Cost')
        ax1.axhline(valid_costs[-1], color='r', linestyle='--', linewidth=1.5,
                   label=f'Final: {valid_costs[-1]:.2f}')
    ax1.set_xlabel('Iteration', fontsize=11)
    ax1.set_ylabel('Path Cost', fontsize=11)
    ax1.set_title('RRT* Cost Convergence\n(Shows asymptotic optimality)', fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Effect of gamma parameter
    ax2 = plt.subplot(2, 3, 2)
    gammas = [r['gamma'] for r in gamma_results]
    lengths = [r['path_length'] for r in gamma_results]
    ax2.plot(gammas, lengths, 'go-', linewidth=2, markersize=10)
    ax2.set_xlabel('Gamma (Rewire Radius Factor)', fontsize=11)
    ax2.set_ylabel('Final Path Length', fontsize=11)
    ax2.set_title('Effect of Rewire Radius Parameter', fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Convergence for different gammas
    ax3 = plt.subplot(2, 3, 3)
    colors = plt.cm.viridis(np.linspace(0, 1, len(gamma_results)))
    for result, color in zip(gamma_results, colors):
        valid = [c for c in result['cost_history'] if c != float('inf')]
        if valid:
            start_idx = len(result['cost_history']) - len(valid)
            ax3.plot(range(start_idx, len(result['cost_history'])), valid,
                    linewidth=2, alpha=0.7, color=color,
                    label=f"γ={result['gamma']:.0f}")
    ax3.set_xlabel('Iteration', fontsize=11)
    ax3.set_ylabel('Path Cost', fontsize=11)
    ax3.set_title('Convergence for Different Gamma Values', fontweight='bold')
    ax3.legend(fontsize=9)
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Statistical comparison (box plot)
    ax4 = plt.subplot(2, 3, 4)
    box_data = [rrt_results, rrt_star_results]
    bp = ax4.boxplot(box_data, labels=['RRT', 'RRT*'], patch_artist=True)
    bp['boxes'][0].set_facecolor('lightblue')
    bp['boxes'][1].set_facecolor('lightgreen')
    ax4.set_ylabel('Path Length', fontsize=11)
    ax4.set_title(f'Statistical Comparison ({num_trials} trials)', fontweight='bold')
    ax4.grid(True, alpha=0.3, axis='y')
    
    # Add mean markers
    means = [np.mean(rrt_results), np.mean(rrt_star_results)]
    ax4.plot([1, 2], means, 'r*', markersize=15, label='Mean', zorder=10)
    ax4.legend()
    
    # Plot 5: Final path visualization
    ax5 = plt.subplot(2, 3, 5)
    for ox, oy, ow, oh in env.obstacles:
        ax5.add_patch(Rectangle((ox, oy), ow, oh, facecolor='gray', 
                                edgecolor='black', linewidth=2))
    if path:
        px, py = zip(*path)
        ax5.plot(px, py, 'b-', linewidth=3, label=f'RRT* Path (L={final_cost:.1f})')
        ax5.scatter(px, py, c='blue', s=40, zorder=5)
    ax5.scatter([start[0]], [start[1]], c='green', s=300, marker='o',
               edgecolors='black', linewidths=2, label='Start', zorder=10)
    ax5.scatter([goal[0]], [goal[1]], c='red', s=300, marker='*',
               edgecolors='black', linewidths=2, label='Goal', zorder=10)
    ax5.set_xlim([0, env.width])
    ax5.set_ylim([0, env.height])
    ax5.set_xlabel('X', fontsize=11)
    ax5.set_ylabel('Y', fontsize=11)
    ax5.set_title('Optimized RRT* Path', fontweight='bold')
    ax5.legend(fontsize=9)
    ax5.grid(True, alpha=0.3)
    ax5.set_aspect('equal')
    
    # Plot 6: Shrinking radius visualization
    ax6 = plt.subplot(2, 3, 6)
    n_nodes = np.arange(1, 500)
    for gamma in [5, 10, 15, 20, 30]:
        d = 2  # 2D space
        radius = gamma * (np.log(n_nodes) / n_nodes)**(1/d)
        # Clip to max step size
        radius = np.minimum(radius, 2.0)
        ax6.plot(n_nodes, radius, linewidth=2, label=f'γ={gamma}')
    ax6.set_xlabel('Number of Nodes', fontsize=11)
    ax6.set_ylabel('Search Radius', fontsize=11)
    ax6.set_title('Shrinking Radius Formula:\nr = γ(log n / n)^(1/d)', fontweight='bold')
    ax6.legend(fontsize=9)
    ax6.grid(True, alpha=0.3)
    
    plt.suptitle('RRT* Comprehensive Analysis', fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()
    
    print("\n" + "="*70)
    print("ANALYSIS COMPLETE")
    print("="*70)
    print("\nKey Insights:")
    print("1. RRT* converges to better solutions over time (asymptotic optimality)")
    print("2. Gamma parameter controls trade-off between:")
    print("   - Higher gamma: More rewiring, better paths, slower")
    print("   - Lower gamma: Less rewiring, faster, potentially suboptimal")
    print("3. Typical gamma range: 10-20 works well for 2D problems")
    print("4. Shrinking radius ensures asymptotic optimality while maintaining efficiency")
    print("5. RRT* consistently produces ~15-30% shorter paths than RRT")
    print("="*70)

# Run the analysis
analyze_rrt_star_convergence()

## Solution to Exercise 3: Dynamic Obstacles (Advanced)

This solution implements RRT for environments with moving obstacles, treating planning as a 3D problem (x, y, t).

In [ ]:
# Exercise 3 Solution: Dynamic RRT for Moving Obstacles

class DynamicObstacle:
    """Moving circular obstacle."""
    def __init__(self, x0, y0, vx, vy, radius):
        self.x0 = x0
        self.y0 = y0
        self.vx = vx
        self.vy = vy
        self.radius = radius
    
    def position_at_time(self, t):
        """Return (x, y) position at time t."""
        return (self.x0 + self.vx * t, self.y0 + self.vy * t)


class DynamicRRTNode:
    """Node in space-time RRT (x, y, t)."""
    def __init__(self, x, y, t):
        self.x = x
        self.y = y
        self.t = t  # Time coordinate
        self.parent = None
        self.cost = 0.0
    
    def __repr__(self):
        return f"Node({self.x:.1f}, {self.y:.1f}, t={self.t:.1f})"


class DynamicRRT:
    """RRT for dynamic environments with moving obstacles."""
    
    def __init__(self, start, goal, environment, dynamic_obstacles,
                 max_iterations=500, step_size=2.0, vehicle_speed=5.0,
                 goal_sample_rate=0.05, max_time=20.0):
        """
        Args:
            start: (x, y) start position
            goal: (x, y) goal position
            environment: Static environment
            dynamic_obstacles: List of DynamicObstacle objects
            vehicle_speed: Speed of vehicle (for time calculation)
            max_time: Maximum time horizon
        """
        self.start = DynamicRRTNode(start[0], start[1], 0.0)
        self.goal_xy = goal
        self.env = environment
        self.dynamic_obstacles = dynamic_obstacles
        self.max_iterations = max_iterations
        self.step_size = step_size
        self.vehicle_speed = vehicle_speed
        self.goal_sample_rate = goal_sample_rate
        self.max_time = max_time
        
        self.nodes = [self.start]
        self.goal_threshold = 2.0
        self.goal_node = None
    
    def is_collision_at_time(self, x, y, t):
        """Check collision with both static and dynamic obstacles at time t."""
        # Static obstacles
        if self.env.is_collision(x, y):
            return True
        
        # Dynamic obstacles
        for obs in self.dynamic_obstacles:
            obs_x, obs_y = obs.position_at_time(t)
            dist = np.sqrt((x - obs_x)**2 + (y - obs_y)**2)
            if dist < obs.radius + 0.5:  # 0.5 margin
                return True
        
        return False
    
    def is_path_collision_free(self, x1, y1, t1, x2, y2, t2, resolution=0.5):
        """Check if space-time path is collision-free."""
        distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        n_points = int(distance / resolution) + 1
        
        for i in range(n_points + 1):
            alpha = i / n_points if n_points > 0 else 0
            x = x1 + alpha * (x2 - x1)
            y = y1 + alpha * (y2 - y1)
            t = t1 + alpha * (t2 - t1)
            
            if self.is_collision_at_time(x, y, t):
                return False
        
        return True
    
    def sample(self):
        """Sample random point in space-time."""
        if np.random.rand() < self.goal_sample_rate:
            # Sample goal with random time
            return self.goal_xy[0], self.goal_xy[1], np.random.uniform(0, self.max_time)
        else:
            x = np.random.uniform(0, self.env.width)
            y = np.random.uniform(0, self.env.height)
            t = np.random.uniform(0, self.max_time)
            return x, y, t
    
    def nearest(self, x, y, t):
        """Find nearest node in tree (Euclidean distance in space only)."""
        min_dist = float('inf')
        nearest_node = None
        
        for node in self.nodes:
            # Only consider nodes in the past (time moves forward)
            if node.t <= t:
                dist = np.sqrt((node.x - x)**2 + (node.y - y)**2)
                if dist < min_dist:
                    min_dist = dist
                    nearest_node = node
        
        return nearest_node
    
    def steer(self, from_node, to_x, to_y, to_t):
        """Create new node considering time based on vehicle speed."""
        # Direction vector
        dx = to_x - from_node.x
        dy = to_y - from_node.y
        spatial_dist = np.sqrt(dx**2 + dy**2)
        
        # Limit spatial distance to step_size
        if spatial_dist > self.step_size:
            dx = dx / spatial_dist * self.step_size
            dy = dy / spatial_dist * self.step_size
            spatial_dist = self.step_size
        
        # Calculate time based on distance and vehicle speed
        travel_time = spatial_dist / self.vehicle_speed
        new_t = from_node.t + travel_time
        
        # Don't exceed time horizon
        if new_t > self.max_time:
            return None
        
        new_node = DynamicRRTNode(from_node.x + dx, from_node.y + dy, new_t)
        new_node.parent = from_node
        new_node.cost = from_node.cost + spatial_dist
        
        return new_node
    
    def plan(self):
        """Run Dynamic RRT algorithm."""
        for i in range(self.max_iterations):
            # Sample
            x_rand, y_rand, t_rand = self.sample()
            
            # Nearest
            nearest_node = self.nearest(x_rand, y_rand, t_rand)
            if nearest_node is None:
                continue
            
            # Steer
            new_node = self.steer(nearest_node, x_rand, y_rand, t_rand)
            if new_node is None:
                continue
            
            # Collision check in space-time
            if self.is_path_collision_free(
                nearest_node.x, nearest_node.y, nearest_node.t,
                new_node.x, new_node.y, new_node.t
            ):
                self.nodes.append(new_node)
                
                # Check if reached goal
                spatial_dist_to_goal = np.sqrt(
                    (new_node.x - self.goal_xy[0])**2 + 
                    (new_node.y - self.goal_xy[1])**2
                )
                
                if spatial_dist_to_goal < self.goal_threshold:
                    # Try to connect to goal
                    goal_time = new_node.t + spatial_dist_to_goal / self.vehicle_speed
                    
                    if (goal_time <= self.max_time and 
                        self.is_path_collision_free(
                            new_node.x, new_node.y, new_node.t,
                            self.goal_xy[0], self.goal_xy[1], goal_time
                        )):
                        goal_node = DynamicRRTNode(self.goal_xy[0], self.goal_xy[1], goal_time)
                        goal_node.parent = new_node
                        goal_node.cost = new_node.cost + spatial_dist_to_goal
                        self.goal_node = goal_node
                        self.nodes.append(goal_node)
                        return self.extract_path(), i + 1
        
        return None, self.max_iterations
    
    def extract_path(self):
        """Extract space-time path."""
        if self.goal_node is None:
            return None
        
        path = []
        current = self.goal_node
        
        while current is not None:
            path.append((current.x, current.y, current.t))
            current = current.parent
        
        path.reverse()
        return path


# Demo: Plan around moving obstacles
print("Dynamic RRT Demo: Planning Around Moving Obstacles")
print("="*70)

# Create environment with fewer static obstacles
dyn_env = Environment(width=50, height=50)
dyn_env.add_obstacle(20, 15, 10, 5)

# Create moving obstacles
dynamic_obs = [
    DynamicObstacle(x0=10, y0=25, vx=1.5, vy=0, radius=3.0),   # Moving right
    DynamicObstacle(x0=40, y0=20, vx=-1.0, vy=0.5, radius=2.5), # Moving left-up
    DynamicObstacle(x0=25, y0=5, vx=0, vy=2.0, radius=2.0),    # Moving up
]

start_dyn = (5, 5)
goal_dyn = (45, 45)

np.random.seed(42)
dyn_rrt = DynamicRRT(start_dyn, goal_dyn, dyn_env, dynamic_obs,
                     max_iterations=800, step_size=2.0, vehicle_speed=5.0,
                     max_time=20.0)
path_dyn, iterations_dyn = dyn_rrt.plan()

if path_dyn:
    print(f"Path found in {iterations_dyn} iterations")
    print(f"Path length: {dyn_rrt.goal_node.cost:.2f}")
    print(f"Travel time: {path_dyn[-1][2]:.2f} seconds")
    print(f"Waypoints: {len(path_dyn)}")
else:
    print("No path found")

# Visualization: Show path at different time snapshots
if path_dyn:
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.flatten()
    
    # Extract spatial path
    path_x = [p[0] for p in path_dyn]
    path_y = [p[1] for p in path_dyn]
    path_t = [p[2] for p in path_dyn]
    
    # Show 6 time snapshots
    time_snapshots = np.linspace(0, path_t[-1], 6)
    
    for idx, t in enumerate(time_snapshots):
        ax = axes[idx]
        
        # Draw static obstacles
        for ox, oy, ow, oh in dyn_env.obstacles:
            ax.add_patch(Rectangle((ox, oy), ow, oh, facecolor='gray',
                                   edgecolor='black', linewidth=2))
        
        # Draw dynamic obstacles at this time
        for obs in dynamic_obs:
            obs_x, obs_y = obs.position_at_time(t)
            circle = plt.Circle((obs_x, obs_y), obs.radius, 
                               color='orange', alpha=0.6, edgecolor='red', linewidth=2)
            ax.add_patch(circle)
            # Draw velocity vector
            ax.arrow(obs_x, obs_y, obs.vx*2, obs.vy*2, 
                    head_width=1, head_length=1, fc='red', ec='red', alpha=0.7)
        
        # Draw full path
        ax.plot(path_x, path_y, 'b--', linewidth=1.5, alpha=0.3, label='Full Path')
        
        # Find vehicle position at this time
        # Find closest waypoint in time
        closest_idx = min(range(len(path_t)), key=lambda i: abs(path_t[i] - t))
        vehicle_x, vehicle_y = path_x[closest_idx], path_y[closest_idx]
        
        # Highlight path up to current time
        past_indices = [i for i, pt in enumerate(path_t) if pt <= t]
        if len(past_indices) > 1:
            ax.plot([path_x[i] for i in past_indices],
                   [path_y[i] for i in past_indices],
                   'b-', linewidth=3, label='Traveled')
        
        # Draw vehicle
        ax.scatter([vehicle_x], [vehicle_y], c='blue', s=200, 
                  marker='D', edgecolors='black', linewidths=2,
                  label='Vehicle', zorder=10)
        
        # Start and goal
        ax.scatter([start_dyn[0]], [start_dyn[1]], c='green', s=200, 
                  marker='o', edgecolors='black', linewidths=2, zorder=9)
        ax.scatter([goal_dyn[0]], [goal_dyn[1]], c='red', s=200, 
                  marker='*', edgecolors='black', linewidths=2, zorder=9)
        
        ax.set_xlim([0, dyn_env.width])
        ax.set_ylim([0, dyn_env.height])
        ax.set_xlabel('X', fontsize=10)
        ax.set_ylabel('Y', fontsize=10)
        ax.set_title(f't = {t:.2f}s', fontsize=12, fontweight='bold')
        if idx == 0:
            ax.legend(fontsize=9, loc='upper left')
        ax.grid(True, alpha=0.3)
        ax.set_aspect('equal')
    
    plt.suptitle('Dynamic RRT: Space-Time Planning\n(Vehicle navigates around moving obstacles)',
                fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

print("\n" + "="*70)
print("Key Concepts:")
print("- Space-time planning: Treats time as a 3rd dimension (x, y, t)")
print("- Time is always moving forward (causality)")
print("- Travel time calculated from spatial distance and vehicle speed")
print("- Collision checking considers obstacle positions at specific times")
print("- Can find paths that wait or time maneuvers to avoid collisions")
print("="*70)

## Additional Explorations

Try these extensions to deepen your understanding:

### 1. Informed RRT*
After finding an initial solution, sample only within an ellipse connecting start and goal with semi-major axis = current path cost. This focuses search on promising regions.

### 2. RRT-Connect
Grow two trees (one from start, one from goal) and try to connect them. Often 2-4× faster than single-tree RRT.

### 3. Kinodynamic RRT
Incorporate vehicle dynamics (steering constraints, velocity limits) by using motion primitives instead of straight-line steering.

### 4. Anytime RRT*
Continue improving the path after finding the first solution, allowing trade-off between computation time and path quality.

### 5. Path Smoothing
Post-process RRT paths using:
- Shortcutting (connecting non-consecutive waypoints if collision-free)
- Bezier curve fitting
- Gradient-based optimization